# Monetary Policy Dashboard Mexico

## 4. Environment Preparation

### 4.1 Importing libraries

(Skip next cell if you already have the following packages installed)

In [1]:
# Objective: Install the Plotly library for interactive visualizations.
!pip install plotly

In [2]:
# Objective: Import necessary libraries and configure graph styles.
import pandas as pd # For data manipulation and analysis in DataFrames.
import numpy as np # For advanced numerical operations, especially with arrays.
import matplotlib.pyplot as plt # For creating static plots.
import seaborn as sns # For high-level statistical visualizations based on matplotlib.
from google.colab import drive # For mounting Google Drive in Colab.
import os # For interacting with the operating system (e.g., file paths).
import plotly.express as px # For quick creation of interactive Plotly graphs.
import plotly.graph_objects as go # For more detailed construction of interactive Plotly graphs.

# Global configuration for default matplotlib figure size.
plt.rcParams["figure.figsize"] = (10, 4)
# Sets the seaborn graph style to 'whitegrid' for better readability.
sns.set_style("whitegrid")

### 4.2 Paths and Global Parameters

In [9]:
# Objective: Mount Google Drive and define the data path.
drive.mount('/content/drive') # Mounts Google Drive to access data files.

# Defines the path to the 'data' folder in Google Drive where the files are located.
#data_path = "/content/drive/MyDrive/Prueba Tecinca MJPG-FINAMEX/data"
data_path = input("Enter the path to the 'data' folder in Google Drive:")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter the path to the 'data' folder in Google Drive:/content/drive/MyDrive/Prueba Tecinca MJPG-FINAMEX/data


## 5. Data Loading and Cleaning

### 5.1 Data Import

In [10]:
# Objective: Define a function to import data from Excel files with dynamic headers.
def importar_desde_header(name_file):
  file_path = os.path.join(data_path,name_file) # Constructs the full file path.
  initial_df = pd.read_excel(file_path, header=None, nrows=25) # Reads the first 25 rows without a header.

  # Finds the indices of key rows.
  date_row_index = initial_df[initial_df[0] == "Fecha"].index[0] # Row containing 'Fecha' (where series start)
  title_row_index = initial_df[initial_df[0] == "Título"].index[0] # Row containing 'Título' (series headers).

  headers = initial_df.iloc[title_row_index].tolist() # Extracts headers from the 'Título' row.

  headers[0] = "date" # Renames the first header to 'date' for standardization.

  df=pd.read_excel(file_path, header=date_row_index) # Reads the Excel using the 'Fecha' row as the actual header.
  df.columns = headers # Assigns the standardized headers to the DataFrame.

  return df # Returns the processed DataFrame.

In [11]:
# Objective: Import and preprocess data from various economic series.

igae_df = importar_desde_header("IGAE.xlsx") # Import IGAE.
igae_df["date"] = pd.to_datetime(igae_df["date"], dayfirst=True, errors="coerce") # Converts 'date' to datetime.

cpi_df = importar_desde_header("INPC.xlsx") # Import CPI.
cpi_df["date"] = pd.to_datetime(cpi_df["date"], dayfirst=True, errors="coerce") # Converts 'date' to datetime.

target_rate_df = importar_desde_header("tasa_objetivo.xlsx") # Import target rate.
target_rate_df = target_rate_df.resample('MS', on='date').mean().reset_index() # Calculates the monthly average of the rate.

exchange_rate_df = importar_desde_header("FIX.xlsx") # Import FIX exchange rate.
exchange_rate_df = exchange_rate_df.resample('MS', on='date').mean().reset_index() # Calculates the monthly average of the exchange rate.

tiiie_rate_df = importar_desde_header("TIIE28.xlsx") # Import TIIE 28-day rate.
tiiie_rate_df = tiiie_rate_df.resample('MS', on='date').mean().reset_index() # Calculates the monthly average of the TIIE.

bonds_rate_df = importar_desde_header("bonosM10.xlsx") # Import 10-year bond rate.
bonds_rate_df["date"] = pd.to_datetime(bonds_rate_df["date"], dayfirst=True, errors="coerce") # Converts 'date' to datetime.

inflation_expectations_df = importar_desde_header("inflacion_e.xlsx") # Import inflation expectations.
inflation_expectations_df["date"] = pd.to_datetime(inflation_expectations_df["date"], dayfirst=True, errors="coerce") # Converts 'date' to datetime.
# Renames the second column to 'value' if it isn't already and converts it to numeric, treating errors as NaN.
if len(inflation_expectations_df.columns) > 1 and inflation_expectations_df.columns[1] != 'value':
    inflation_expectations_df.rename(columns={inflation_expectations_df.columns[1]: 'value'}, inplace=True)
inflation_expectations_df['value'] = pd.to_numeric(inflation_expectations_df['value'], errors='coerce') # Converts 'value' to numeric.

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

In [12]:
# Objective: Review and confirm the periodicity of the target_rate (should be monthly).
target_rate_df.head(13) # Displays the first 13 rows of the 'target_rate_df' DataFrame.

,date,Tasa objetivo
0,2008-01-01,7.500000
1,2008-02-01,7.500000
2,2008-03-01,7.500000
3,2008-04-01,7.500000
4,2008-05-01,7.500000
5,2008-06-01,7.583333
6,2008-07-01,7.858696
7,2008-08-01,8.130952
8,2008-09-01,8.250000
9,2008-10-01,8.250000


In [13]:
# Objective: Review which index each column of the CPI corresponds to.
cpi_df.head(1) # Displays the first row of the 'cpi_df' DataFrame.

,date,"IPC Por objeto del gasto Nacional, I n d i c e G e n e r a l","Subíndices subyacente y complementarios, Precios al consumidor (INPC), Subyacente","Subíndices subyacente y complementarios, Precios al consumidor (INPC), No Subyacente"
0,2000-01-01,44.93083,48.96917,33.997198


In [14]:
# Objective: Create a DataFrame for general CPI.
cpi_general = cpi_df.iloc[:, :2].copy() # Selects the first two columns (date and general CPI).
cpi_general.columns = ["date", "value"] # Renames columns to 'date' and 'value'.
cpi_general.head(1) # Displays the first row of the 'cpi_general' DataFrame.

,date,value
0,2000-01-01,44.93083


In [15]:
# Objective: Create a DataFrame for core CPI.
cpi_core = cpi_df.iloc[:,[0,2]].copy() # Selects the date column and the core CPI column.
cpi_core.columns = ["date", "value"] # Renames columns to 'date' and 'value'.
cpi_core.head(1) # Displays the first row of the 'cpi_core' DataFrame.

,date,value
0,2000-01-01,48.96917


### 5.2 Homogenization

In [16]:
# Objective: Standardize and prepare time series.

def prepare_series(df):
    df.columns = ["date", "value"] # Renames columns to 'date' and 'value'.
    df = df.sort_values("date").set_index("date") # Sorts by date and sets 'date' as index.
    df = df.asfreq("MS") # Ensures monthly frequency (fills in missing months).
    df['value'] = pd.to_numeric(df['value'], errors='coerce') # Converts 'value' to numeric, errors to NaN.
    return df

igae_df = prepare_series(igae_df) # Applies to IGAE.
igae_df=igae_df.reset_index("date") # Resets the 'date' index as a regular column.
cpi_general = prepare_series(cpi_general) # Applies to general CPI.
cpi_core = prepare_series(cpi_core) # Applies to core CPI.
bonds_rate_df = prepare_series(bonds_rate_df) # Applies to bond rates.
target_rate_df = prepare_series(target_rate_df) # Applies to target rate.
exchange_rate_df = prepare_series(exchange_rate_df) # Applies to exchange rate.
tiiie_rate_df = prepare_series(tiiie_rate_df) # Applies to TIIE.
inflation_expectations_df = prepare_series(inflation_expectations_df) # Applies to inflation expectations.

### 5.3 Construction of derived variables

In [17]:
# Objective: Calculate inflation and economic activity indicators.

# Annual inflation (year-over-year percentage change).
cpi_general["inflation_yoy"] = cpi_general["value"].pct_change(12) * 100 # General inflation YoY.
cpi_core["core_inflation_yoy"] = cpi_core["value"].pct_change(12) * 100 # Core inflation YoY.

# Moving averages of general inflation to identify trends.
cpi_general["inflation_mm3"] = cpi_general["inflation_yoy"].rolling(3).mean() # 3-month moving average.
cpi_general["inflation_mm6"] = cpi_general["inflation_yoy"].rolling(6).mean() # 6-month moving average.

# Economic activity indicators.
igae_df["igae_yoy"] = igae_df["value"].pct_change(12) * 100 # Annual IGAE growth.
igae_df["igae_momentum"] = igae_df["value"].diff(3) # IGAE momentum (change over 3 months).

In [18]:
# Objective: Create a unified DataFrame for financial conditions.
# 'inner merge' is used to include only dates where all indicators are present.
financial_conditions_df = exchange_rate_df.rename(columns={'value': 'TC_FIX'}) # Renames 'value' column to 'TC_FIX'.
financial_conditions_df = financial_conditions_df.merge(tiiie_rate_df.rename(columns={'value': 'TIIE28'}), on='date') # Adds TIIE28.
financial_conditions_df = financial_conditions_df.merge(bonds_rate_df.rename(columns={'value': 'BondsM10'}), on='date') # Adds BondsM10.
financial_conditions_df = financial_conditions_df.merge(inflation_expectations_df.rename(columns={'value': 'Inflation_Expectation'}), on='date') # Adds Inflation_Expectation.

In [19]:
# Objective: Calculate key financial rates for analysis.

# Real Ex-ante Rate (TIIE28 - 12-month Inflation Expectation).
financial_conditions_df['Inflation_Expectation'] = pd.to_numeric(financial_conditions_df['Inflation_Expectation'], errors='coerce') # Ensures the column is numeric, converts errors to NaN.
financial_conditions_df['Real_Rate'] = financial_conditions_df['TIIE28'] - financial_conditions_df['Inflation_Expectation'] # Calculates the real rate.

# Yield Curve Slope (Spread between Long and Short Term: BondsM10 - TIIE28).
financial_conditions_df['Curve_Slope'] = financial_conditions_df['BondsM10'] - financial_conditions_df['TIIE28'] # Calculates the curve slope.

# Accumulated Exchange Rate Change (Normalized to detect shocks: percentage change over 6 months).
financial_conditions_df['FX_Var_6M'] = financial_conditions_df['TC_FIX'].pct_change(6) * 100 # Calculates the percentage change of FX over 6 months.

In [20]:
# Objective: Calculate Z-Scores to normalize indicators and create a Financial Conditions Index (FCI).
for col in ['Real_Rate', 'Curve_Slope', 'FX_Var_6M']: # Iterates over financial indicators.
    financial_conditions_df[f'{col}_Z'] = (financial_conditions_df[col] - financial_conditions_df[col].mean()) / financial_conditions_df[col].std() # Calculates the Z-Score.

# Calculates the Aggregate Financial Conditions Index (FCI).
# The Z-Score of the Curve_Slope is inverted (multiplied by -1) because a negative slope indicates greater stress.
financial_conditions_df['Conditions_Index'] = (
    financial_conditions_df['Real_Rate_Z'] + # Z-Score of the Real Rate.
    (financial_conditions_df['Curve_Slope_Z'] * -1) + # Inverted Z-Score of the Curve Slope.
    financial_conditions_df['FX_Var_6M_Z'] # Z-Score of Exchange Rate Vulnerability.
) / 3 # Averages the Z-Scores.
financial_conditions_df=financial_conditions_df.reset_index("date") # Resets the 'date' index as a regular column.

## 6. Inflation Diagnosis

### 6.1 Headline vs. core inflation

In [21]:
# Objective: Visualize general vs. core inflation.

# Combines general and core inflation series into a single DataFrame.
combined_inflation_df = pd.DataFrame({
    'General': cpi_general["inflation_yoy"],
    'Core': cpi_core["core_inflation_yoy"]
})

combined_inflation_df = combined_inflation_df.reset_index() # Resets the 'date' index as a column.

# Creates an interactive line chart with Plotly Express.
fig = px.line(
    combined_inflation_df,
    x='date',
    y=['General', 'Core'], # Plots both columns as separate lines.
    title="Annual Inflation (2018 Base): General vs. Core",
    labels={
        "value": "Inflation (%)", # Y-axis label.
        "variable": "Inflation Type" # Legend title.
    }
)

# Adds a horizontal line for the inflation target (3%).
fig.add_hline(y=3, line_dash="dash", line_color="black", annotation_text="Inflation Target (3%)", annotation_position="bottom right")

fig.show()

#### Where is inflation heading in Mexico?
Global inflation has moderated since 2023 and is projected to remain at 5% by 2025; however, its recent fluctuations are alarming. The real focus of attention is core inflation, which not only refuses to decline but has shown a slight upward trend since March 2025, moving further away from the 3% target.

This situation is a warning sign, as it indicates that the price increases are no longer a temporary shock to specific products (such as gasoline or food) but have spread to the rest of the economy. This lack of a clear decline toward the target confirms a persistent inflationary environment, which reduces Banxico's room for short-term rate cuts without jeopardizing price stability.

### 6.2 Signs of Inflationary Trends

In [22]:
# Objective: Visualize observed inflation and its trend signals (moving averages).

# Combines observed inflation and its moving averages into a DataFrame.
combined_inflation_df = pd.DataFrame({
    'Observed Inflation (Effective)': cpi_general["inflation_yoy"],
    'Short-Term Trend (3M)': cpi_general["inflation_mm3"],
    'Structural Inertia (6M)': cpi_general["inflation_mm6"]
})

combined_inflation_df = combined_inflation_df.reset_index() # Resets the 'date' index as a column.

# Creates an interactive line chart with Plotly Express.
fig = px.line(
    combined_inflation_df,
    x='date',
    y=['Observed Inflation (Effective)', 'Short-Term Trend (3M)', 'Structural Inertia (6M)'], # Plots all three series.
    title="Inflation and Trend Signal",
    labels={
        "value": "Inflation (%)" # Y-axis label.
    }
)

# Adds a horizontal line for the inflation target (3%).
fig.add_hline(y=3, line_dash="dash", line_color="black", annotation_text="Inflation Target (3%)", annotation_position="bottom right")

fig.show()

#### Is disinflation progressing or has it stalled?

Inflation is showing signs of cooling in the short term, with the 3-month moving average trading below the 6-month moving average since August 2025.

However, the disinflation process has reached a phase of structural stagnation at approximately 4%, with the 6-month moving average failing to break below 3%, a level last observed in May 2020.

This resistance to converging to the target suggests market acceptance of new price levels, necessitating sustained monetary tightening to force the eventual convergence to 3%.

## 7. Economic Activity Diagnostic

### 7.1 Level and cycle of economic activity

In [23]:
# Objective: Visualize the annual growth of the Global Economic Activity Index (IGAE).

# Combines the annual growth series of IGAE into a DataFrame.
combined_igae_df = pd.DataFrame({
    'date': igae_df["date"],
    'Annual IGAE Growth': igae_df["igae_yoy"]
})

# Creates an interactive line chart with Plotly Express.
fig = px.line(
    combined_igae_df,
    x='date',
    y=['Annual IGAE Growth'],
    title='Global Economic Activity Index (IGAE) <br><sup>Seasonally Adjusted Series</sup>',
    labels={
        "value": "Variation - Adjusted Series (%)" # Y-axis label.
    }
)

# Adds a horizontal line at 0% to indicate the stagnation threshold.
fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Stagnation Threshold (0%)", annotation_position="bottom right")

fig.show()

#### Is the economy stagnating?

Looking at the end of 2025, the current turning point is the loss of momentum. After the normalization of 2022-2023, the economy failed to maintain sustained growth above the historical average of 2% or 3%.

Economic activity hovers around zero, indicating stagnation rather than contraction; consequently, it lacks the necessary "power" to drive prices down through supply. This confirms that inflation is "subject" to cost factors and not to excessive economic dynamism.

In this context, the economy operates in a fragile equilibrium, vulnerable to shocks that could push it into recession.

### 7.2 Momentum of economic activity

In [24]:
# Objective: Visualize the short-term momentum of IGAE (change over 3 months).

# Combines the IGAE momentum series into a DataFrame.
combined_igae_momentum_df = pd.DataFrame({
    'date': igae_df["date"],
    'Real Momentum': igae_df["igae_momentum"]
})

# Creates an interactive line chart with Plotly Express.
fig = px.line(
    combined_igae_momentum_df, # Uses the correct DataFrame for momentum.
    x='date',
    y=['Real Momentum'],
    title='Short-Term Velocity: IGAE (Δ 3M) <br><sup> Calculation based on seasonally adjusted series</sup>',
    labels={
        "value": "Quarterly Adjusted Variation (%)" # Y-axis label.
    }
)

# Adds a horizontal line at 0% to indicate the stagnation zone.
fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Stagnation Zone (0%)", annotation_position="bottom right")

fig.show()

#### Is there a risk of stagflation?
As of the end of 2025 and the beginning of 2026, the momentum of the IGAE (Global Indicator of Economic Activity) has stagnated near zero, reflecting an economy with no capacity for acceleration. This contrasts with core inflation, which remains elevated, suggesting that price pressures stem not from excess demand but from costs and structural inertia.

The fact that growth is zero prevents inflation from falling due to a lack of consumption, trapping the economy in a scenario of stagflationary risk. Given the negative momentum observed in September 2025 and the frequency of adjustment cycles (approximately 6-9 months), the data suggest that the impact of monetary policy has already materialized, limiting the benefit of maintaining restrictive rates in the face of structurally low growth.

## 8. Financial Conditions and Monetary Stance

### 8.1 Inflation and monetary policy rate

In [25]:
# Objective: Compare General Inflation with Banxico's Target Rate.

# Combines general inflation and the target rate series into a DataFrame.
combined_df = pd.DataFrame({
    'General Inflation (Annual Var. %)': cpi_general["inflation_yoy"],
    "Target Rate (Monthly Average)" : target_rate_df["value"]
})

combined_df = combined_df.reset_index() # Resets the 'date' index as a column.

# Renames the column 'index' to 'date' if it exists.
if 'index' in combined_df.columns:
      combined_df.rename(columns={'index': 'date'}, inplace=True)

# Creates an interactive line chart with Plotly Express.
fig = px.line(
    combined_df,
    x='date',
    y=['General Inflation (Annual Var. %)', 'Target Rate (Monthly Average)'],
    title='Monetary Control: Inflation vs. Banxico Target Rate',
    labels={
        "value": "Percentage (%)" # Y-axis label.
    }
)

# Adds a horizontal line for the inflation target (3%).
fig.add_hline(y=3, line_dash="dash", line_color="black", annotation_text="Inflation Target (3%)", annotation_position="bottom right")

fig.show()

#### How restrictive is the monetary policy stance?
The gap between the interest rate and inflation remains wide as we approach the end of 2025, indicating a clearly restrictive monetary policy stance in real terms.

Although the target (nominal) rate is beginning to decline, inflation is also falling, maintaining a positive real rate that continues to dampen economic activity. The gradual evolution of the rate confirms the lag with which monetary policy operates and its precautionary nature.

Overall, the graphical evidence suggests a restrictive stance in the face of persistent inflation and weak economic growth.

### 8.3 Additional Financial Indicators

In [26]:
# Objective: Compare Banxico's Target Rate with the Exchange Rate.

# Creates a temporary DataFrame with the Target Rate and Exchange Rate.
temp_combined_df = pd.DataFrame({
    "Target Rate (Monthly Average)" : target_rate_df["value"],
    "MXN/USD Exchange Rate (Monthly Average)" : exchange_rate_df["value"]
})

combined_inflation_df = temp_combined_df.reset_index() # Resets the 'date' index as a column.

# Renames the column 'index' to 'date' if it exists.
if 'index' in combined_inflation_df.columns:
    combined_inflation_df.rename(columns={'index': 'date'}, inplace=True)

# Creates an interactive line chart with Plotly Express.
fig = px.line(
    combined_inflation_df,
    x='date',
    y=["Target Rate (Monthly Average)", "MXN/USD Exchange Rate (Monthly Average)"],
    title='Monetary Decoupling: Rate Cuts in a Stabilized Depreciation Environment',
    labels={
        "value": "Percentage (%)" # Y-axis label.
    }
)

# Adds a horizontal line for the inflation target (3%) as a reference.
fig.add_hline(y=3, line_dash="dash", line_color="black", annotation_text="Inflation Target (3%)", annotation_position="bottom right")
# Improves hover information.
fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Value: %{y:.2f}<br>Difference vs. Previous Month: %{dy:.2f}"
)
fig.show()

#### Is the exchange rate constraining monetary policy?

A decoupling is observed between the exchange rate trajectory and the interest rate, as the latter continues to adjust downward despite the peso's depreciation.

The dollar's stabilization at a new level suggests that Banxico has accepted a different exchange rate equilibrium without triggering additional inflationary pressures. The absence of a rebound in inflation expectations indicates that pass-through has been limited.

In this context, monetary policy appears to prioritize weak economic activity over exchange rate risks.

## 9. Integration of Macroeconomic Signals

### 9.1 Construction of synthetic indicators (Z-score)

In [27]:
# Objective: Visualize individual signals and the aggregate financial conditions index.

fig = go.Figure() # Creates a Plotly figure.

# Adds traces for Monetary Rigor, Curve Inversion, and Exchange Rate Vulnerability (Z-Scores).
for col, name in zip(['Real_Rate_Z', 'Curve_Slope_Z', 'FX_Var_6M_Z'],
                       ['Monetary Rigor', 'Curve Inversion', 'Exchange Rate Vulnerability']):
    fig.add_trace(go.Scatter(x=financial_conditions_df['date'], y=financial_conditions_df[col],
                                    name=name, mode='lines'))

# Adds the Aggregate Financial Conditions Index (FCI) with shading.
fig.add_trace(go.Scatter(x=financial_conditions_df['date'], y=financial_conditions_df['Conditions_Index'],
                                name='Aggregate Index (FCI)', line=dict(color='black', width=3),
                                fill='tozeroy', fillcolor='rgba(0,0,0,0.1)'))

# Configures the graph layout.
fig.update_layout(
    title='<b>Financial Conditions Signal',
    xaxis_title='Date', yaxis_title='Standard Deviations (Z-Score)',
    legend_title='Components', template='plotly_white'
)
# Improves hover information.
fig.update_traces(
    hovertemplate="<b>%{scroll_zoom}</b><br>Z-Score: %{y:.2f}<br>Deviation: %{y:+.2f}σ"
)

# Adjusts graph height and adds explanatory annotations.
fig.update_layout(height=650,
    margin=dict(t=80, b=150, l=80, r=80),
    annotations=[
        dict(
            xref="paper", yref="paper",
            x=0, y=-0.25,
            showarrow=False,
            text=(
                "• <b>FCI > 0:</b> <b>Restrictive</b> Conditions. The financial system acts as an economic brake.<br>" +
                "• <b>FCI < 0:</b> <b>Loose</b> Conditions. The financial system facilitates dynamism and credit."
            ),
            align="left",
            font=dict(size=13, color="black")
        )
    ]
)
fig.show()

#### Are financial conditions restrictive?
The yield curve has crossed into negative territory by the end of 2025, marking the end of the period of extreme financial stress seen in 2024. This improvement is mainly due to the easing of monetary tightening and the stabilization of the exchange rate, allowing the market to feel less constrained even before Banxico implements deep rate cuts.

Although the inverted yield curve continues to indicate caution regarding long-term growth, the overall outlook reflects systemic relief. This chart confirms that the excessive tightening has ended, giving Banxico the necessary leeway in 2026 to soften its stance and avoid stifling economic activity (IGAE) without reigniting inflationary risks.

### 9.2 Correlations and consistency of signals

In [28]:
# Objective: Analyze the contribution of each indicator to the current level of financial stress.

# 1. Gets the latest available data on financial conditions.
latest_data = financial_conditions_df.tail(1) # Latest record of the DataFrame.

# 2. Prepares a DataFrame with the impact (Z-Score) of each indicator.
# The Z-Score of the Curve Slope is inverted (multiplied by -1).
contribution = pd.DataFrame({
    'Indicator': ['Monetary Rigor', 'Curve Inversion', 'Exchange Rate Vulnerability'],
    'Impact (Z-Score)': [
        latest_data['Real_Rate_Z'].values[0],
        latest_data['Curve_Slope_Z'].values[0] * -1, # Inverted Curve Slope.
        latest_data['FX_Var_6M_Z'].values[0]
    ]
})

# 3. Creates a horizontal bar chart to visualize the impact.
fig_contrib = px.bar(
    contribution,
    x='Impact (Z-Score)',
    y='Indicator',
    orientation='h',
    color='Impact (Z-Score)', # Colors the bars according to their value.
    color_continuous_scale='RdBu_r', # Color scale: Red (restrictive), Blue (loose).
    title='<b>What explains the current stress level? (latest data as of Dec 2025)</b>',
    labels={'Impact (Z-Score)': 'Impact (Z-Score) (σ)'},
    template='plotly_white'
)

fig_contrib.add_vline(x=0, line_color="black", line_width=2) # Adds a reference line at 0.

fig_contrib.show()

#### Have financial conditions ceased to be restrictive?
The easing of monetary tightening and the stabilization of exchange rate vulnerability indicate that conditions are no longer putting significant pressure on the real economy.

Although the inverted yield curve suggests caution regarding long-term growth, overall systemic stress has lessened.

Overall, the graphical evidence suggests that Banxico has more room to soften its policy stance without generating immediate financial imbalances.

In [29]:
# Objective: Visualize the decision trajectory (Financial Stress vs. Economic Growth).


# 1. Combines financial conditions with IGAE momentum and sorts by date.
df_decision = financial_conditions_df.merge(igae_df[['date', 'igae_momentum']], on='date', how='left') # Combines DataFrames.
df_decision = df_decision.sort_values('date') # Sorts by date.

# 2. Creates an interactive scatter plot.
fig_scatter = go.Figure() # Initializes the figure.

# Adds points to the graph, colored by year and without a trajectory line.
fig_scatter.add_trace(go.Scatter(
    x=df_decision['Conditions_Index'],
    y=df_decision['igae_momentum'],
    mode='markers', # Shows only markers, not lines.
    name='Temporal Trajectory',
    marker=dict( # Marker configuration.
        size=10,
        color=df_decision['date'].dt.year, # Color of markers according to the year (numeric value).
        colorscale='RdBu_r', # Color scale.
        showscale=True,
        colorbar=dict(title="Year") # Legend for the color scale.
    ),
    customdata=df_decision['date'].dt.strftime('%b %Y'), # Data for hover (month and year).
    hovertemplate=( # Text format for hover.
        "<b>Month: %{customdata}</b><br>" +
        "Financial Stress: %{x:.2f}σ<br>" +
        "IGAE Growth: %{y:.2f}%<br>" +
        "<extra></extra>" # Removes extra hover information.
    )
))

# 3. Configures the decision quadrant layout.
fig_scatter.update_layout(
    title='<b>Decision Trajectory: Stress vs. Growth (Feb 2026)</b>',
    xaxis_title='Financial Stress Index (Z-Score)',
    yaxis_title='Economic Growth (IGAE Momentum)',
    template='plotly_white',
    height=650
)

# Adds quadrant lines at zero for both axes.
fig_scatter.add_vline(x=0, line_dash="dash", line_color="black")
fig_scatter.add_hline(y=0, line_dash="dash", line_color="black")

# Adds text labels for the "IDEAL" and "ALERT" zones.
fig_scatter.add_annotation(x=-1.5, y=2, text="<b>IDEAL ZONE</b>", showarrow=False, font=dict(color="green"))
fig_scatter.add_annotation(x=1.5, y=-2, text="<b>ALERT ZONE</b>", showarrow=False, font=dict(color="red"))

fig_scatter.show()

#### What has been Banxico's decision-making trajectory?

The strongest correlation with the decline in the overall stress index by the end of 2025 stems from the improved exchange rate and the easing of monetary tightening.

The outlook for February 2026 indicates that last year's financial “problems,” such as the strong dollar and market stress, are now under control. However, the economy has stalled, while prices are still resisting falling to their ideal level. Given this stagnation, Banxico has the opportunity to lower interest rates to provide some relief to economic activity without jeopardizing the country's stability.

## 10. Evaluation of future expected scenarios

### 10.1 Scenario Matrix
Considering the previous results

In [30]:
# Create scenario data based on your Z-Scores and IGAE analysis
scenarios_data = {
    "Scenario": [
        "Base (Soft Landing)",
        "Alert (Stagflation)",
        "Optimistic (Recovery)"
    ],
    "Financial Condition (FCI)": [
        "Loose / Neutral (-0.4σ)", # Current value according to the dashboard (end of 2025)
        "Restrictive (> 1.0σ)",   # Statistical threshold for stress alert
        "Very Loose (< -1.0σ)"      # Conditions of abundant liquidity
    ],
    "IGAE Momentum": [
        "Stagnant (0%)",         # Observed behavior in recent months
        "Contraction (-1.5%)",    # Decline that would force to stop cuts to avoid devaluation
        "Rebound (> 1.0%)"         # Growth above current potential
    ],
    "Suggested Banxico Action": [
        "Cut 25bp",           # Fine-tuning to avoid economic strangulation
        "Monetary Pause",        # To prevent capital flight due to country risk
        "Cut 50bp"           # Accelerated normalization due to solid growth
    ],
    "Probability": [
        "65%", # Based on the current trend of stress decompression
        "25%", # Tail risk due to external factors or volatility
        "10%"  # Unlikely given the observed structural stagnation
    ]
}

df_scenarios = pd.DataFrame(scenarios_data)

# Create the visual table in Plotly with executive format
fig_tabla = go.Figure(data=[go.Table(
    header=dict(
        values=[f"<b>{col}</b>" for col in df_scenarios.columns],
        fill_color='navy',
        align='center',
        font=dict(color='white', size=13),
        height=35
    ),
    cells=dict(
        values=[df_scenarios[col] for col in df_scenarios.columns],
        fill_color=['white', 'white', 'white', 'white', 'white'],
        align='left',
        font=dict(size=12),
        height=30
    ))
])

fig_tabla.update_layout(
    title="<b>Decision Matrix: Scenarios for the February 2026 Board Meeting</b><br><sup>Sustained based on Financial Conditions Index and IGAE Momentum</sup>",
    margin=dict(l=20, r=20, t=60, b=20)
)

fig_tabla.show()

#### Risk Balance Analysis (Expected Scenarios)
The matrix above summarizes the roadmap for the next monetary policy decision.

1. ICF (Z-Scores): The standard scale is used, where 0 is the historical average.

-0.4σ indicates that stress is below average but not extreme.

>1.0σ is considered "Restrictive" (one standard deviation above).

2. IGAE Momentum: Real growth thresholds are defined.

0% is stagnation.

-1.5% is a clear sign of a technical recession.

3. Banxico Action: Based on the standard magnitude of movements (25 or 50 basis points).

The Baseline Scenario (65% probability) is based on the easing of financial stress observed in the Dashboard, suggesting that the risk of maintaining a high rate (“over-restraining”) is now greater than the risk of an inflationary spike due to the exchange rate.

However, persistent underlying inflation makes it impossible to rule out the optimistic scenario of aggressive cuts (50 bps). Therefore, the path of least resistance is a gradual 25 basis point cut, contingent on the IGAE momentum not falling into negative territory, which would trigger the Alert Scenario and require a pause to assess the stability of growth.

### 10.2 Sensitivity Graph: Actual Rate vs. Neutral Range


In [31]:
# Define Banxico's neutral range
# Instead of manual values, we use the statistics from YOUR data
neutral_rate_mean = financial_conditions_df['Real_Rate'].mean()
rate_std = financial_conditions_df['Real_Rate'].std()

# We define the range according to your own database (Mean +/- 1 standard deviation)
neutral_rate_min = neutral_rate_mean - rate_std
neutral_rate_max = neutral_rate_mean + rate_std

fig_sens = go.Figure()

# 1. Neutral Zone (The "Should Be")
# Used as a reference to know if policy stimulates or brakes.
fig_sens.add_hrect(
    y0=neutral_rate_min, y1=neutral_rate_max,
    fillcolor="green", opacity=0.2, line_width=0,
    annotation_text="Neutral Rate Range (Equilibrium)",
    annotation_position="top left"
)

# 2. Real Ex-ante Rate (The Reality)
# It is the Nominal Rate minus inflation expectations.
# If it is above the green zone, Banxico is "braking."
fig_sens.add_trace(go.Scatter(
    x=financial_conditions_df['date'].tail(24),
    y=financial_conditions_df['Real_Rate'].tail(24),
    mode='lines+markers',
    name='Current Real Rate',
    line=dict(color='firebrick', width=3),
    hovertemplate="Date: %{x}<br>Real Rate: %{y:.2f}%<extra></extra>"
))

# 3. Format and Analysis Question
fig_sens.update_layout(
    title='<b>Is current monetary policy restrictive?</b>',
    xaxis_title='Analysis Date',
    yaxis_title='Real Rate (%)',
    template='plotly_white',
    annotations=[dict(
        x=financial_conditions_df['date'].iloc[-1],
        y=financial_conditions_df['Real_Rate'].iloc[-1],
        text="<b>Restriction Gap</b>",
        showarrow=True,
        arrowhead=1,
        ay=10,
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="firebrick",
        borderwidth=1
    )]
)

fig_sens.show()

#### Is the current rate historically high for the period analyzed?
When comparing the current real rate against the historical average of our series, we observe that we are above the average. This demonstrates, with the data presented in this dashboard, that the current policy stance is more restrictive than the normal behavior of recent years. This reinforces the signal that there is room to cut rates without exceeding the historical parameters of the Mexican economy.

## 11. Conclusion and Recommendations

### Expected Monetary Stance

Inflation is undergoing a slow and costly convergence process. Monetary policy (Chart 5) remains restrictive on average for the month to counteract the inertia of core inflation (Chart 1). However, this “solution” has already cooled the economic engine, bringing the IGAE's momentum to stagnant levels (Chart 4). The immediate future suggests a gradual reduction in interest rates to avoid a recession, accepting that inflation will take longer than anticipated to reach 3%.

1. State of Financial Conditions
Stress Easing: The Aggregate Financial Conditions Index (ICF) is currently in negative territory, confirming that the period of extreme financial constraint observed in 2024 has ended.

Controlled Exchange Rate Vulnerability: The fixed exchange rate has shifted from being the main driver of stress at the beginning of 2025 to becoming a factor of stability and flexibility in 2026.

Persistence of Tightness: Despite the general easing, “Monetary Tightness” (Real Interest Rate) remains the component with the greatest relative pressure, acting as the last preventive brake of current policy.

2. Economic Cycle Diagnosis (Trajectory)
Position in the Quadrant: The economy is moving within the “Ideal Zone,” characterized by low financial stress; however, it is dangerously close to the stagnation axis.

IGAE Momentum: With real growth stagnating near 0%, the trajectory suggests that restrictive monetary policy has already completed its cycle of inflation control and now faces diminishing returns on economic activity.

3. Strategic Recommendation
Room for Rate Cuts: The combination of a stable peso, anchored inflation expectations, and an accommodative financial system gives Banxico the necessary leeway to continue reducing the target rate.

Growth Priority: To prevent the stagnation of the IGAE (Global Indicator of Economic Activity) from becoming a formal contraction (Alert Zone), it is imperative to normalize the residual “Monetary Tightness” identified in the contribution analysis.